# Estimate Time Request for Key Project Proposal

This notebook summarizes the results of simulating the proposed follow-up strategy for targets from the different surveys, and computes the overal total observing time requested on LCO facilities. 

For details of the calculations of total numbers of targets selected from each survey, and the corresponding observing time calculations, please see notebooks:
* ./gaia/estimated_number_gaia_events2.ipynb
* ./ztf/estimated_number_ztf_events2.ipynb
* ./ogle/estimated_number_bulge_events2.ipynb

## Summary of time for each element of observations

Here we summarize the findings of the above notebooks, in terms of the expected numbers of targets and the observing time required to complete the proposed strategy on each instrument class, as required by the proposal.  

Note that there may be slight differences in the total time estimated, as the previous notebooks allowed decimal numbers of targets whereas here we require integers. 

In [1]:
import numpy as np

In [2]:
class TargetSet:
    def __init__(self):
        self.survey = None
        self.survey_years = []
        self.n_planet_events_per_year = 0
        self.n_stellar_events_per_year = 0
        self.n_bh_events_per_year = 0
        
        self.sinistro_time_per_planet = 0
        self.sinistro_time_per_star = 0
        self.sinistro_time_per_bh = 0
        
        self.floyds_frac = 0.0
        self.floyds_time_per_target = 3.83 # hrs
        
        self.time_per_caustic = 6.0*2.0 # hrs, with two caustic crossings per event
        self.time_per_anomaly = 4.0 # hrs
        self.vis_fraction = 0.0
    
    def calc_total_n_events(self):
        """Calculate the total number of events over the lifetime of the survey operations"""
        self.n_planet_events = self.n_planet_events_per_year * len(self.survey_years)
        self.n_stellar_events = self.n_stellar_events_per_year * len(self.survey_years)
        self.n_bh_events = self.n_bh_events_per_year * len(self.survey_years) 
        
        print('Expect ~'+str(self.n_planet_events_per_year)+' planetary events per year, and '+str(self.n_planet_events)+' over the remaining survey lifetime')
        print('Expect ~'+str(self.n_stellar_events_per_year)+' stellar events per year, and '+str(self.n_stellar_events)+' over the remaining survey lifetime')
        print('Expect ~'+str(self.n_bh_events_per_year)+' stellar remnant events per year, and '+str(self.n_bh_events)+' over the remaining survey lifetime')
      
    def calc_sinistro_time(self):
        """Calculate the time required on the 1m/Sinistro network for different categories of events"""
        
        self.sinistro_time_planet_per_year = self.sinistro_time_per_planet * self.n_planet_events_per_year
        self.sinistro_time_stellar_per_year = self.sinistro_time_per_star * self.n_stellar_events_per_year
        self.sinistro_time_bh_per_year = self.sinistro_time_per_bh * self.n_bh_events_per_year
        print('Time on 1m/Sinistro network required for planetary events per year = '+str(self.sinistro_time_planet_per_year)+'hrs')
        print('Time on 1m/Sinistro network required for stellar events per year = '+str(self.sinistro_time_stellar_per_year)+'hrs')
        print('Time on 1m/Sinistro network required for stellar remnant events per year = '+str(self.sinistro_time_bh_per_year)+'hrs')
        
        self.sinistro_time_planet = []
        self.sinistro_time_stellar = []
        self.sinistro_time_bh = []
        self.sinistro_time = []
        for i,year in enumerate(self.survey_years):
            self.sinistro_time_planet.append(self.sinistro_time_planet_per_year)
            self.sinistro_time_stellar.append(self.sinistro_time_stellar_per_year)
            self.sinistro_time_bh.append(self.sinistro_time_bh_per_year)
            
            time_year = self.sinistro_time_planet_per_year \
                        + self.sinistro_time_stellar_per_year \
                        + self.sinistro_time_bh_per_year
            print('Total time on 1m/Sinistro in year '+str(year)+': '+str(time_year))
            self.sinistro_time.append(time_year)
            
    def calc_floyds_time(self, planets=True, stellar=True):
        """Calculate the time required for 2m/FLOYDS spectroscopy"""
        
        self.floyds_bh_events_per_year = int(round(self.floyds_frac*self.n_bh_events_per_year,0))
        print('Number of stellar remnant events bright enough for FLOYDS per year = '+str(self.floyds_bh_events_per_year))
        self.floyds_time_bh_per_year = int(round((self.floyds_bh_events_per_year * self.floyds_time_per_target),0))
        print('Time on 2m/FLOYDS network required for stellar remnant events per year = '+str(self.floyds_time_bh_per_year)+'hrs')

        if planets:
            self.floyds_planet_events_per_year = int(round(self.floyds_frac*self.n_planet_events_per_year,0))
            print('Number of planetary events bright enough for FLOYDS per year = '+str(self.floyds_planet_events_per_year))
            
            self.floyds_time_planet_per_year = int(round((self.floyds_planet_events_per_year * self.floyds_time_per_target),0))
            print('Time on 2m/FLOYDS network required for planetary events per year = '+str(self.floyds_time_planet_per_year)+'hrs')
        
        if stellar:
            self.floyds_stellar_events_per_year = int(round(self.floyds_frac*self.n_stellar_events_per_year,0))
            print('Number of stellar events bright enough for FLOYDS per year = '+str(self.floyds_stellar_events_per_year))
        
            self.floyds_time_stellar_per_year = int(round((self.floyds_stellar_events_per_year * self.floyds_time_per_target),0))
            print('Time on 2m/FLOYDS network required for stellar events per year = '+str(self.floyds_time_stellar_per_year)+'hrs')
    
        self.floyds_time_planet = []
        self.floyds_time_stellar = []
        self.floyds_time_bh = []
        self.floyds_time = []
        for i,year in enumerate(self.survey_years):
            if planets:
                self.floyds_time_planet.append(self.floyds_time_planet_per_year)
            else:
                self.floyds_time_planet.append(0.0)
            
            if stellar:
                self.floyds_time_stellar.append(self.floyds_time_stellar_per_year)
            else:
                self.floyds_time_stellar.append(0.0)
                                                
            self.floyds_time_bh.append(self.floyds_time_bh_per_year)
            
            time_year = self.floyds_time_bh_per_year
            if planets:
                time_year += self.floyds_time_planet_per_year
            if stellar:
                time_year += self.floyds_time_stellar_per_year
                
            self.floyds_time.append(time_year)
            print('Total time on 2m/FLOYDS in year '+str(year)+': '+str(time_year))

    def calc_2m_imagers_time(self, planets=True, stellar=True):
        """Estimate of time required in hours on the 2m/MusCAT or 2m/Spectral network to 
        observe caustic crossings for stellar & planetary binary events in RAPID RESPONSE MODE"""

        self.time_rr_stellar_events = []
        self.time_rr_planet_events = []
        self.time_2m_imagers = []
        for i,year in enumerate(self.survey_years):
            time_year = 0.0
            if stellar:
                n_stellar_per_year = int(round((self.n_stellar_events_per_year * self.vis_fraction),0))
                time_stellar_year = n_stellar_per_year * self.time_per_caustic
                self.time_rr_stellar_events.append(time_stellar_year)
                time_year += time_stellar_year
                print('Time on 2m/imagers for '+str(n_stellar_per_year)+' stellar events in year '+str(year)+': '+str(time_stellar_year))
            else:
                self.time_rr_stellar_events.append(0.0)
            
            if planets:
                n_planet_per_year = int(round((self.n_planet_events_per_year * self.vis_fraction),0))
                time_planet_year = n_planet_per_year * self.time_per_caustic
                self.time_rr_planet_events.append(time_planet_year)
                time_year += time_planet_year
                print('Time on 2m/imagers for '+str(n_planet_per_year)+' planetary events in year '+str(year)+': '+str(time_planet_year))
            else:
                self.time_rr_planet_events.append(0.0)
                
            self.time_2m_imagers.append(time_year)
            print('Total time on 2m/imagers in RR mode in year '+str(year)+': '+str(time_year))
    
    def calc_1m_rr_time(self, planets=True, stellar=True):
        """Time required in Rapid Reponse mode from the 1m network is equivalent to the fraction of the time 
        dedicated to observing anomalies and caustic crossings that isn't already covered by the 2m. 
        
        It is only calculated for the planetary and stellar binary events to be followed from each survey. 
        
        """
        
        # The fraction of the Rapid Response time from the 1m network
        rr_fraction = 1.0 - self.vis_fraction
        
        self.time_rr_stellar_events_1m = []
        self.time_rr_planet_events_1m = []
        self.time_rr_1m_imagers = []
        for i,year in enumerate(self.survey_years):
            time_year = 0.0
            if stellar:
                n_stellar_per_year = int(round((self.n_stellar_events_per_year * rr_fraction),0))
                time_stellar_year = n_stellar_per_year * self.time_per_caustic
                self.time_rr_stellar_events_1m.append(time_stellar_year)
                time_year += time_stellar_year
                print('Rapid response time on 1m/imagers for '+str(n_stellar_per_year)+' stellar events in year '+str(year)+': '+str(time_stellar_year))
            else:
                self.time_rr_stellar_events_1m.append(0.0)
            
            if planets:
                n_planet_per_year = int(round((self.n_planet_events_per_year * rr_fraction),0))
                time_planet_year = n_planet_per_year * self.time_per_caustic
                self.time_rr_planet_events_1m.append(time_planet_year)
                time_year += time_planet_year
                print('Rapid response time on 1m/imagers for '+str(n_planet_per_year)+' planetary events in year '+str(year)+': '+str(time_planet_year))
            else:
                self.time_rr_planet_events_1m.append(0.0)
                
            self.time_rr_1m_imagers.append(time_year)
            print('Total time on 1m/imagers in RR mode in year '+str(year)+': '+str(time_year))
        
        
    def calc_time_totals(self):
        self.sinistro_time_total = np.array(self.sinistro_time).sum()
        self.floyds_time_total = np.array(self.floyds_time).sum()
        self.time_2m_imagers_total = np.array(self.time_2m_imagers).sum()
        
        self.sinistro_1m_queue = np.zeros(3)
        for i,year in enumerate(self.survey_years):
            self.sinistro_1m_queue[i] = self.sinistro_time[i] - self.time_rr_1m_imagers[i]
            print('Year '+str(year)+' Sinistro total '+str(self.sinistro_time[i]))
            print(' -> Sinistro queue mode '+str(self.sinistro_1m_queue[i]))
            print(' -> Sinistro RR mode '+str(self.time_rr_1m_imagers[i]))
        

### Gaia Targets

The simulation considers LCO observations of stellar remnant and stellar (most likely single-lens) events detected by Gaia over its last two years of operation (2023-2025). 

Note that this does not preclude observing a stellar or planetary binary of interest if one is discovered, but for the purpose of the proposal I have aimed to be conservative. 

In [3]:
gaia_targets = TargetSet()

In [4]:
# Note the 'planets' category here is used to refer to any binary event, since Gaia binaries are likely to be outside
# the Bulge and therefore high priority, while the stellar category here refers to single-lens stellar mass lenses. 
# The 'planet' events per year are subtracted from the overall total number of events expected per year to avoid 
# double counting
gaia_targets.survey_years = [2023, 2024]
gaia_targets.n_planet_events_per_year = 9
gaia_targets.n_stellar_events_per_year = 92 - gaia_targets.n_planet_events_per_year
gaia_targets.n_bh_events_per_year = 7
gaia_targets.calc_total_n_events()      


Expect ~9 planetary events per year, and 18 over the remaining survey lifetime
Expect ~83 stellar events per year, and 166 over the remaining survey lifetime
Expect ~7 stellar remnant events per year, and 14 over the remaining survey lifetime


In [5]:
# Estimates of time required in hours on the 1m/Sinistro network to observe the different categories of events
gaia_targets.sinistro_time_per_planet = 8+12
gaia_targets.sinistro_time_per_star = 4.5
gaia_targets.sinistro_time_per_bh = 6.2
gaia_targets.calc_sinistro_time()


Time on 1m/Sinistro network required for planetary events per year = 180hrs
Time on 1m/Sinistro network required for stellar events per year = 373.5hrs
Time on 1m/Sinistro network required for stellar remnant events per year = 43.4hrs
Total time on 1m/Sinistro in year 2023: 596.9
Total time on 1m/Sinistro in year 2024: 596.9


In [6]:
# Estimates of time required in hours on the 2m/FLOYDS network to observe the different categories of events
gaia_targets.floyds_frac = 0.994 # Fraction of events likely to be bright enough for FLOYDS
gaia_targets.calc_floyds_time(stellar=False)


Number of stellar remnant events bright enough for FLOYDS per year = 7
Time on 2m/FLOYDS network required for stellar remnant events per year = 27hrs
Number of planetary events bright enough for FLOYDS per year = 9
Time on 2m/FLOYDS network required for planetary events per year = 34hrs
Total time on 2m/FLOYDS in year 2023: 61
Total time on 2m/FLOYDS in year 2024: 61


In [7]:
# Estimate of time required in hours on the 2m/MusCAT or 2m/Spectral network to 
# observe caustic crossings for stellar binary events in RAPID RESPONSE MODE
# Note 'stellar' for this function refers to all single stellar events
gaia_targets.vis_fraction = 8.0 / 24.0
gaia_targets.calc_2m_imagers_time(stellar=False)



Time on 2m/imagers for 3 planetary events in year 2023: 36.0
Total time on 2m/imagers in RR mode in year 2023: 36.0
Time on 2m/imagers for 3 planetary events in year 2024: 36.0
Total time on 2m/imagers in RR mode in year 2024: 36.0


In [8]:
# Estimate the remaining Rapid Response time required from the 1m network
gaia_targets.calc_1m_rr_time(stellar=False)

Rapid response time on 1m/imagers for 6 planetary events in year 2023: 72.0
Total time on 1m/imagers in RR mode in year 2023: 72.0
Rapid response time on 1m/imagers for 6 planetary events in year 2024: 72.0
Total time on 1m/imagers in RR mode in year 2024: 72.0


### ZTF Targets

Similarly to Gaia events, the simulation assumes that ZTF is most likely to detect stellar remnant and single-lens stellar events.  

In [9]:
ztf_targets = TargetSet()

In [22]:
ztf_targets.survey_years = [2023, 2024, 2025]
ztf_targets.n_planet_events_per_year = 0.34
ztf_targets.n_stellar_events_per_year = 14
ztf_targets.n_bh_events_per_year = 1
ztf_targets.calc_total_n_events()

Expect ~0.34 planetary events per year, and 1.02 over the remaining survey lifetime
Expect ~14 stellar events per year, and 42 over the remaining survey lifetime
Expect ~1 stellar remnant events per year, and 3 over the remaining survey lifetime


In [23]:
# Estimates of time required in hours on the 1m/Sinistro network to observe the different categories of events
ztf_targets.sinistro_time_per_planet = 6+12
ztf_targets.sinistro_time_per_star = 4.2
ztf_targets.sinistro_time_per_bh = 9.2
ztf_targets.calc_sinistro_time()


Time on 1m/Sinistro network required for planetary events per year = 6.12hrs
Time on 1m/Sinistro network required for stellar events per year = 58.800000000000004hrs
Time on 1m/Sinistro network required for stellar remnant events per year = 9.2hrs
Total time on 1m/Sinistro in year 2023: 74.12
Total time on 1m/Sinistro in year 2024: 74.12
Total time on 1m/Sinistro in year 2025: 74.12


In [24]:
# Estimates of time required in hours on the 2m/FLOYDS network to observe the different categories of events
ztf_targets.floyds_frac = 0.995 # Fraction of events likely to be bright enough for FLOYDS
ztf_targets.calc_floyds_time()

Number of stellar remnant events bright enough for FLOYDS per year = 1
Time on 2m/FLOYDS network required for stellar remnant events per year = 4hrs
Number of planetary events bright enough for FLOYDS per year = 0
Time on 2m/FLOYDS network required for planetary events per year = 0hrs
Number of stellar events bright enough for FLOYDS per year = 14
Time on 2m/FLOYDS network required for stellar events per year = 54hrs
Total time on 2m/FLOYDS in year 2023: 58
Total time on 2m/FLOYDS in year 2024: 58
Total time on 2m/FLOYDS in year 2025: 58


In [25]:
# Estimate of time required in hours on the FTN 2m/MusCAT to 
# observe caustic crossings for stellar binary events in RAPID RESPONSE MODE
ztf_targets.vis_fraction = 8.0 / 24.0
ztf_targets.calc_2m_imagers_time(stellar=False)

Time on 2m/imagers for 0 planetary events in year 2023: 0.0
Total time on 2m/imagers in RR mode in year 2023: 0.0
Time on 2m/imagers for 0 planetary events in year 2024: 0.0
Total time on 2m/imagers in RR mode in year 2024: 0.0
Time on 2m/imagers for 0 planetary events in year 2025: 0.0
Total time on 2m/imagers in RR mode in year 2025: 0.0


In [26]:
# Estimate the remaining Rapid Response time required from the 1m network
ztf_targets.calc_1m_rr_time(stellar=False)

Rapid response time on 1m/imagers for 0 planetary events in year 2023: 0.0
Total time on 1m/imagers in RR mode in year 2023: 0.0
Rapid response time on 1m/imagers for 0 planetary events in year 2024: 0.0
Total time on 1m/imagers in RR mode in year 2024: 0.0
Rapid response time on 1m/imagers for 0 planetary events in year 2025: 0.0
Total time on 1m/imagers in RR mode in year 2025: 0.0


### Bulge Targets from the extended survey region only

The simulation considers only events from a rough annulus around the central Bulge, within the OGLE extended survey region but not including those fields covered by KMTNet at high cadence.  It is hard to justify time for follow-up observations within that high cadence zone. 

The Bulge simulation considers planetary and stellar binary events only and long-timescale (>100d) single-lens events. 

In [27]:
bulge_targets = TargetSet()

In [41]:
bulge_targets.survey_years = [2023, 2024, 2025]
bulge_targets.n_stellar_events_per_year = 0 # No single-lens stellar events will be followed
bulge_targets.n_planet_events_per_year = 9 # Includes all binary events, planetary, BD, stellar companions
bulge_targets.n_bh_events_per_year = 6
bulge_targets.calc_total_n_events()


Expect ~9 planetary events per year, and 27 over the remaining survey lifetime
Expect ~0 stellar events per year, and 0 over the remaining survey lifetime
Expect ~6 stellar remnant events per year, and 18 over the remaining survey lifetime


In [42]:
# Estimates of time required in hours on the 1m/Sinistro network to observe the different categories of events
bulge_targets.sinistro_time_per_planet = 21.1
#bulge_targets.sinistro_time_per_star = 18
bulge_targets.sinistro_time_per_bh = 26.4
bulge_targets.calc_sinistro_time()

Time on 1m/Sinistro network required for planetary events per year = 189.9hrs
Time on 1m/Sinistro network required for stellar events per year = 0hrs
Time on 1m/Sinistro network required for stellar remnant events per year = 158.39999999999998hrs
Total time on 1m/Sinistro in year 2023: 348.29999999999995
Total time on 1m/Sinistro in year 2024: 348.29999999999995
Total time on 1m/Sinistro in year 2025: 348.29999999999995


In [43]:
# Estimates of time required in hours on the 2m/FLOYDS network to observe the different categories of events
bulge_targets.floyds_frac = 0.121 # Fraction of events likely to be bright enough for FLOYDS
bulge_targets.calc_floyds_time()

Number of stellar remnant events bright enough for FLOYDS per year = 1
Time on 2m/FLOYDS network required for stellar remnant events per year = 4hrs
Number of planetary events bright enough for FLOYDS per year = 1
Time on 2m/FLOYDS network required for planetary events per year = 4hrs
Number of stellar events bright enough for FLOYDS per year = 0
Time on 2m/FLOYDS network required for stellar events per year = 0hrs
Total time on 2m/FLOYDS in year 2023: 8
Total time on 2m/FLOYDS in year 2024: 8
Total time on 2m/FLOYDS in year 2025: 8


In [44]:
# Estimate of time required in hours on the FTS 2m/Spectral or MusCAT to 
# observe caustic crossings for planetary and stellar binary events in RAPID RESPONSE MODE
bulge_targets.vis_fraction = 0.42
bulge_targets.calc_2m_imagers_time()


Time on 2m/imagers for 0 stellar events in year 2023: 0.0
Time on 2m/imagers for 4 planetary events in year 2023: 48.0
Total time on 2m/imagers in RR mode in year 2023: 48.0
Time on 2m/imagers for 0 stellar events in year 2024: 0.0
Time on 2m/imagers for 4 planetary events in year 2024: 48.0
Total time on 2m/imagers in RR mode in year 2024: 48.0
Time on 2m/imagers for 0 stellar events in year 2025: 0.0
Time on 2m/imagers for 4 planetary events in year 2025: 48.0
Total time on 2m/imagers in RR mode in year 2025: 48.0


In [45]:
# Estimate the remaining Rapid Response time required from the 1m network
bulge_targets.calc_1m_rr_time()

Rapid response time on 1m/imagers for 0 stellar events in year 2023: 0.0
Rapid response time on 1m/imagers for 5 planetary events in year 2023: 60.0
Total time on 1m/imagers in RR mode in year 2023: 60.0
Rapid response time on 1m/imagers for 0 stellar events in year 2024: 0.0
Rapid response time on 1m/imagers for 5 planetary events in year 2024: 60.0
Total time on 1m/imagers in RR mode in year 2024: 60.0
Rapid response time on 1m/imagers for 0 stellar events in year 2025: 0.0
Rapid response time on 1m/imagers for 5 planetary events in year 2025: 60.0
Total time on 1m/imagers in RR mode in year 2025: 60.0


## Maximal targets observed

If all planned observations execute completely, we should characterize the following target set over the whole project lifetime.  

Note that stellar remnant events are sufficiently long in timescale that Gaia and ZTF may both detect the same candidates.  For that reason, I've avoided double-counting them, though potentially this may be an under-estimate.  


In [47]:
total_planets = gaia_targets.n_planet_events + ztf_targets.n_planet_events + bulge_targets.n_planet_events
total_stars = gaia_targets.n_stellar_events + ztf_targets.n_stellar_events + bulge_targets.n_stellar_events
total_bh = max(gaia_targets.n_bh_events, ztf_targets.n_bh_events)
total_bh = max(total_bh, bulge_targets.n_bh_events)

print(str(total_planets)+' planetary events')
print(str(total_stars)+' stellar binary events')
print(str(total_bh)+' stellar remnant event candidates')


46.019999999999996 planetary events
208 stellar binary events
18 stellar remnant event candidates


## Total Observing Time Required

Various alternative strategies are considered here, as the total is determined by the full range of the targets we expect to study.   

In [48]:
# Calculate the time required in total for each instrument class for each TargetSet
gaia_targets.calc_time_totals()
ztf_targets.calc_time_totals()
bulge_targets.calc_time_totals()

Year 2023 Sinistro total 596.9
 -> Sinistro queue mode 524.9
 -> Sinistro RR mode 72.0
Year 2024 Sinistro total 596.9
 -> Sinistro queue mode 524.9
 -> Sinistro RR mode 72.0
Year 2023 Sinistro total 74.12
 -> Sinistro queue mode 74.12
 -> Sinistro RR mode 0.0
Year 2024 Sinistro total 74.12
 -> Sinistro queue mode 74.12
 -> Sinistro RR mode 0.0
Year 2025 Sinistro total 74.12
 -> Sinistro queue mode 74.12
 -> Sinistro RR mode 0.0
Year 2023 Sinistro total 348.29999999999995
 -> Sinistro queue mode 288.29999999999995
 -> Sinistro RR mode 60.0
Year 2024 Sinistro total 348.29999999999995
 -> Sinistro queue mode 288.29999999999995
 -> Sinistro RR mode 60.0
Year 2025 Sinistro total 348.29999999999995
 -> Sinistro queue mode 288.29999999999995
 -> Sinistro RR mode 60.0


### Maximal Strategy

Queue-mode photometry and spectroscopy of all observable stellar and stellar remnant lenses from Gaia, ZTF and the Bulge as well as planetary events from the extended survey region in the Bulge (outside the KMTNet high cadence zone).

Rapid response observations on the 2m imagers including MusCAT for planetary anomalies and stellar caustic crossings.

In [57]:
print('Time per instrument class per year:')
sinistro_time = np.zeros(3)
sinistro_queue = np.zeros(3)
sinistro_rr = np.zeros(3)
floyds_time = np.zeros(3)
ft_imagers_time = np.zeros(3)
for i,year in enumerate([2023,2024,2025]):
    for targets in [gaia_targets, ztf_targets, bulge_targets]:
        if year in targets.survey_years:
            sinistro_time[i] += targets.sinistro_time[i]
            sinistro_queue[i] += targets.sinistro_1m_queue[i]
            sinistro_rr[i] += targets.time_rr_1m_imagers[i]
            floyds_time[i] += targets.floyds_time[i]
            ft_imagers_time[i] += targets.time_2m_imagers[i]
            
    print('Year '+str(year)+' 1m/Sinistro time = '+str(int(round(sinistro_time[i],0)))+'hrs')
    print('-> Queue time = '+str(int(round(sinistro_queue[i],0)))+'hrs')
    print('-> Rapid response time = '+str(int(round(sinistro_rr[i],0)))+'hrs')
    print('Year '+str(year)+' 2m/FLOYDS time = '+str(int(round(floyds_time[i],0)))+'hrs')
    print('Year '+str(year)+' 2m/imagers time (RR mode) = '+str(int(round(ft_imagers_time[i],0)))+'hrs')
    print('\n')
    
print('Time per instrument over project lifetime:')
sinistro_total = int(round(np.array(sinistro_time).sum(),0))
sinistro_queue_total = int(round(sinistro_queue.sum(),0))
sinistro_rr_total = int(round(sinistro_rr.sum(),0))
floyds_total = int(round(np.array(floyds_time).sum(),0))
ft_imagers_total = int(round(np.array(ft_imagers_time).sum(),0))
project_total = sinistro_total + floyds_total + ft_imagers_total

print('1m/Sinistro: '+str(sinistro_total)+'hrs')
print('-> Queue mode: '+str(sinistro_queue_total)+'hrs')
print('-> Rapid response mode: '+str(sinistro_rr_total)+'hrs')
print('2m/FLOYDS: '+str(floyds_total)+'hrs')
print('2m/imagers: '+str(ft_imagers_total)+'hrs (Rapid Response mode)\n')

print('Overall project request = '+str(project_total)+'hrs')

Time per instrument class per year:
Year 2023 1m/Sinistro time = 1019hrs
-> Queue time = 887hrs
-> Rapid response time = 132hrs
Year 2023 2m/FLOYDS time = 127hrs
Year 2023 2m/imagers time (RR mode) = 84hrs


Year 2024 1m/Sinistro time = 1019hrs
-> Queue time = 887hrs
-> Rapid response time = 132hrs
Year 2024 2m/FLOYDS time = 127hrs
Year 2024 2m/imagers time (RR mode) = 84hrs


Year 2025 1m/Sinistro time = 422hrs
-> Queue time = 362hrs
-> Rapid response time = 60hrs
Year 2025 2m/FLOYDS time = 66hrs
Year 2025 2m/imagers time (RR mode) = 48hrs


Time per instrument over project lifetime:
1m/Sinistro: 2461hrs
-> Queue mode: 2137hrs
-> Rapid response mode: 324hrs
2m/FLOYDS: 320hrs
2m/imagers: 216hrs (Rapid Response mode)

Overall project request = 2997hrs


### Stellar remnant candidate observations only

Queue-mode photometry of stellar remnant candidates from Gaia, ZTF and the Bulge using the 1m network, plus FLOYDS spectroscopy of those that reach bright enough peak magnitudes. 

No rapid response observations.  No 2m imaging observations. 

In [ ]:
sinistro_time_per_year = gaia_targets.sinistro_time_bh_per_year \
                            + ztf_targets.sinistro_time_bh_per_year \
                            + bulge_targets.sinistro_time_bh_per_year
sinistro_time_total = gaia_targets.sinistro_time_bh \
                            + ztf_targets.sinistro_time_bh \
                            + bulge_targets.sinistro_time_bh

floyds_time_per_year = gaia_targets.floyds_time_bh_per_year \
                            + ztf_targets.floyds_time_bh_per_year \
                            + bulge_targets.floyds_time_bh_per_year
floyds_time_total = gaia_targets.floyds_time_bh \
                            + ztf_targets.floyds_time_bh \
                            + bulge_targets.floyds_time_bh

project_total = sinistro_time_total + floyds_time_total
print('Time per instrument class per year:')
print('1m/Sinistro: '+str(sinistro_time_per_year)+'hrs')
print('2m/FLOYDS: '+str(floyds_time_per_year)+'hrs')

print('Time per instrument over project lifetime:')
print('1m/Sinistro: '+str(sinistro_time_total)+'hrs')
print('2m/FLOYDS: '+str(floyds_time_total)+'hrs')

print('Overall project request = '+str(project_total)+'hrs')

## Stellar remnant and planet candidate observations only

Photometry of stellar remnant candidates from Gaia, ZTF and the Bulge using the 1m network. 

Photometry of planetary events in the Bulge extended survey region. 

FLOYDS spectroscopy of those that reach bright enough peak magnitudes.

Rapid response imaging from 2m network for planetary anomalies. 

In [ ]:
sinistro_time_per_year = gaia_targets.sinistro_time_planet_per_year + gaia_targets.sinistro_time_bh_per_year\
                            + ztf_targets.sinistro_time_planet_per_year + ztf_targets.sinistro_time_bh_per_year \
                            + bulge_targets.sinistro_time_planet_per_year + bulge_targets.sinistro_time_bh_per_year
sinistro_time_total = gaia_targets.sinistro_time_planet +  gaia_targets.sinistro_time_bh\
                            + ztf_targets.sinistro_time_planet + ztf_targets.sinistro_time_bh\
                            + bulge_targets.sinistro_time_planet + bulge_targets.sinistro_time_bh

floyds_time_per_year = gaia_targets.floyds_time_bh_per_year + gaia_targets.floyds_time_planet_per_year\
                            + ztf_targets.floyds_time_bh_per_year + ztf_targets.floyds_time_planet_per_year\
                            + bulge_targets.floyds_time_bh_per_year + bulge_targets.floyds_time_planet_per_year
floyds_time_total = gaia_targets.floyds_time_bh + gaia_targets.floyds_time_planet \
                            + ztf_targets.floyds_time_bh + ztf_targets.floyds_time_planet \
                            + bulge_targets.floyds_time_bh + bulge_targets.floyds_time_planet

# 2m imagers are only used for planetary anomalies
FT_imagers_time_per_year = gaia_targets.time_2m_imagers_planet_per_year \
                            + ztf_targets.time_2m_imagers_planet_per_year  \
                            + bulge_targets.time_2m_imagers_planet_per_year 
FT_imagers_time_total = gaia_targets.time_2m_imagers_planet  \
                            + ztf_targets.time_2m_imagers_planet \
                            + bulge_targets.time_2m_imagers_planet

project_total = sinistro_time_total + floyds_time_total + FT_imagers_time_total
print('Time per instrument class per year:')
print('1m/Sinistro: '+str(sinistro_time_per_year)+'hrs')
print('2m/FLOYDS: '+str(floyds_time_per_year)+'hrs')
print('2m/imagers: '+str(FT_imagers_time_per_year)+'hrs\n')

print('Time per instrument over project lifetime:')
print('1m/Sinistro: '+str(sinistro_time_total)+'hrs')
print('2m/FLOYDS: '+str(floyds_time_total)+'hrs')
print('2m/imagers: '+str(FT_imagers_time_total)+'hrs (Rapid Response mode)\n')

print('Overall project request = '+str(project_total)+'hrs')